# Defining the field $\mathbb{F}_q$ and the polynomial ring $\mathbb{F}_q[T]$

Define base field:

In [60]:
# Define base field by prime power:
q = 2

Calculate the field $F_q$:

($g$ is a generator of the multiplicative group of $F_q$)

In [61]:
Fq = GF(q,'g')
g = Fq.gen()

Define the polynomial rings $\mathbb{F}_q[T]$, $\mathbb{F}_q[T,X]$, and $\mathbb{F}_q[T][X]$:

In [62]:
FqT.<T> = Fq[]
FFqT.<T> = FunctionField(Fq)
FqTX.<T,X> = Fq[]
FFqT_X.<X> = FFqT[]
sep_TX = FqTX.hom([T,X],FFqT_X)

# Helper Functions

## Polynomial algebra and number theory

Factors of a polynomial:

In [63]:
def prime_factors(p):
    return set(pr for pr,n in list(p.factor()))
def cross_multiply(l):
    if len(l) == 0:
        return set(1)
    else:
        return set(p*q for p in l[0] for q in cross_multiply(l[1:]))
def all_factors(p):
    list_of_powers = [set(pr^i for i in range(n+1)) for pr,n in list(p.factor())]
    return cross_multiply(list_of_powers)
def is_monicList(l):
    for p in l:
        if p != 0:
            return p.is_monic()

Euler $\varphi$ function:

In [64]:
from functools import reduce

def qNorm(p):
    return q^p.degree()
def product(iterable):
    return reduce(operator.mul, iterable, 1)
def EulerPhi(n, p):
    pfs = prime_factors(p)
    pf_norms = [qNorm(pf) for pf in pfs]
    return qNorm(p)^n * product(1-pfn^(-n) for pfn in pf_norms)

Useful lists modulo a polynomial:

In [65]:
@cached_function
def numsModDegree(n):
    if n == 0:
        return [0]
    elif n > 0:
        highMonos = [c*FqT(T)^(n-1) for c in Fq]
        return [hm+num for hm in highMonos for num in numsModDegree(n-1)]
    else:
        raise ValueError
def numsMod(p):
    p = FqT(p)
    return numsModDegree(p.degree())

@cached_function
def coprimesMod(p):
    p = FqT(p)
    return [num for num in numsMod(p) if p.gcd(num) == 1]

@cached_function
def pairsModDegree(n):
    nums = numsModDegree(n)
    return [(n1,n2) for n1 in nums for n2 in nums]
def pairsMod(p):
    p = FqT(p)
    return pairsModDegree(p.degree())

def reducedPairsDegree(n):
    return [(n1,n2) for n1,n2 in pairsModDegree(n) if n1 != 0 or n2 != 0]
def reducedPairs(p):
    p = FqT(p)
    return reducedPairsDegree(p.degree())

@cached_function
def monicPairsDegree(n):
    return [pair for pair in reducedPairsDegree(n) if is_monicList(pair)]
def monicPairs(p):
    p = FqT(p)
    return monicPairsDegree(p.degree())

# @cached_function
def ppairsDegree(n):
    redPairs = reducedPairsDegree(n)
    return [(p1,p2) for p1 in redPairs for p2 in redPairs if p1[0]+p2[0] != 0 or p1[1]+p2[1] != 0]
def ppairs(p):
    p = FqT(p)
    return ppairsDegree(p.degree())

@cached_function
def monicPPairsDegree(n):
    return [(p1,p2) for p1,p2 in ppairsDegree(n) if is_monicList(p1+p2)] # + is tuple concatenation
def monicPPairs(p):
    p = FqT(p)
    return monicPPairsDegree(p.degree())

@cached_function
def cusps(p):
    p = FqT(p)
    return [(p1,p2) for p1,p2 in monicPairs(p) if p.gcd(p1).gcd(p2) == 1]

def my_xgcd(l):
    if len(l) < 2:
        raise ValueError
    elif len(l) == 2:
        return l[0].xgcd(l[1])
    else: # len(l) > 2
        xgcd_rest = my_xgcd(l[1:])
        pgcd = l[0].xgcd(xgcd_rest[0])
        return pgcd[:2] + tuple(pgcd[2]*xgcd_rest[i] for i in range(1,len(l)))
    
@cached_function
def biCusps(p):
    p = FqT(p)
    cc = cusps(p)
    xgcd_coeffs = [my_xgcd(c+(p,))[1:] for c in cc]
    return [(c,(-x[1],x[0])) for c,x in zip(cc,xgcd_coeffs)]

# biCusps(T^2+T)

## Drinfeld module functions

Frobenius endomorphism and ring of skew polynomials:

In [66]:
frob = FqTX.hom([FqTX(T)^q,FqTX(X)^q])
frob
SkewTX.<tau> = FqTX['tau',frob]
# SkewTX
# phiaT = T +T*tau +tau^2; phiaT(T)

$\varphi^{\pi A}_p(x)$ for a polynomial $p \in \mathbb{F}_q[T]$, output in $\mathbb{F}_q(T)[X]$:

In [67]:
phi_hom = FqT.hom([T+tau],SkewTX)

@cached_function
def phiA(p):
    return sep_TX(phi_hom(FqT(p)).operator_eval(X))

# (phiA(T^2+T)/phiA(T+1)/phiA(T)*phiA(1))(1/X).parent()

Most primitive factor of $\varphi_p^{\pi A}(x)$, output in $\mathbb{F}_q(T)[X]$:

In [68]:
@cached_function
def primPhiA(p):
    p = FqT(p)
    primes = list(prime_factors(p))
    return FFqT_X(phiA_inclExcl_div(p,primes))

def phiA_inclExcl_div(p,l):
    if len(l) == 0:
        return phiA(p)
    else:
        first = l[0]
        rest = l[1:]
        return phiA_inclExcl_div(p,rest)/phiA_inclExcl_div(p/first,rest)

# primPhiA(T^2+T) == phiA(T^2+T)/phiA(T+1)/phiA(T)*phiA(1)
# primPhiA(T^2+T).parent()

$\pi e_A \left(\frac{r}{N}\right) = e_{\pi a} \left(\frac{\pi r}{N}\right)$

In [69]:
@cached_function
def FqT_adj(N):
    # ext = FFqT_X.quotient(primPhiA(N), 'e1')
    why.<e> = FFqT[]
    poly = primPhiA(N)(e)
    ext.<e> = FFqT.extension(poly)
    return ext, e

# FqT_N, e = FqT_adj(T^2+T); FqT_N

In [70]:
@cached_function
def eA(r,N):
    ext, e1 = FqT_adj(N)
    return phiA(r % N)(e1)


# eA(T^3+T^2-T, T^2+T)
# eA(T^3+T^2-T, T^2+T).parent()

## Expansions of Eisenstein series

Integrality factor

In [71]:
@cached_function
def int_factor(N):
    return FqT(N).radical()

# int_factor(T^2+T).parent()

Expansion of an Eisenstein series at $\infty$

In [72]:
# X-expansion of an Eisenstein series at infinity to order 'omax', exclusive; currently only works if omax <= |N|
# 1/(eA(r2,N)+phiA(r1,1/X))
@cached_function
def E2SeriesInfinity(r,omax,N):
    r1,r2 = r
    r1 %= N; r2 %= N
    deg, lc = r1.degree(), r1.lc()

    radical = int_factor(N)

    if omax > qNorm(N):
        print(f"Warning: incomplete E-series expansion since omax, {omax}, is greater than the norm of N, {qNorm(N)}.")
    
    if deg == -1:
        # r1 == 0 mod N
        return radical / eA(r2,N)
    else:
        frac = 1 -X^(q^deg) * (eA(r2,N)+phiA(r1)(1/X)) / lc
        # return from fraction field to polynomial ring
        fracfield = frac.parent()
        section = fracfield.coerce_map_from(fracfield.ring()).section()
        frac = section(frac)
        imax = omax - q^deg
        inexp = 0
        for i in range(imax+1):
            inexp += frac^i
        return radical * X^(q^deg) * inexp / lc +O(X^omax)

# E2SeriesInfinity((1,T), 6, T^2)
# E2SeriesInfinity((1,T), 6, T^2).parent()

Series expansion of a product of Eisenstein series at a cusp

In [73]:
@cached_function
def E2Series(r,biCusp,N):
    ab,cd = biCusp; a,b = ab; c,d = cd
    R = [((a*r1+c*r2) % N, (b*r1+d*r2) % N) for r1,r2 in r]
    omax = (len(r) * qNorm(N) / (q+1) +1).floor()
    series = product(E2SeriesInfinity(rbc,omax,N) for rbc in R) +O(X^omax)
    return series

def E2Series_list(r,biCusp,N,deg):
    series = E2Series(r,biCusp,N)
    l = series.list()
    # return l
    if len(l) >= deg:
        return l[:deg]
    else: # len(l) < deg
        return l +[0]*(deg-len(l))

# E2Series([(1,0),(T,2),(2,T+1)], ((0,1),(-1,0)), T^2+T)
# E2Series_list(((1,0),(T,2),(2,T+1)), ((0,1),(-1,0)), T^2+T, 15)


# Main function

In [81]:
from itertools import combinations_with_replacement
import time
import datetime
log_level = 1
mPrintFreq = 5

def ifPrintTemp(n,*args):
    if log_level >= n:
        print("\r", *args, sep='', end='')

def stats(N):
    start = time.time()

    N = FqT(N)
    ifPrintTemp(1, "Beginning calculation of statistics for", N)

    ifPrintTemp(1, "Forming set of Eisenstein pairs")
    ReddPairs = reducedPairs(N)
    Ppairs = list(combinations_with_replacement(ReddPairs,2))

    ifPrintTemp(1, "Calculating t-expansions of Eisenstein pairs")
    series_cutoff = ceil(2 * qNorm(N) / (q+1))
    c_lists = [
        [E2Series_list(r,bc,N,series_cutoff) for bc in biCusps(N)]
        for r in Ppairs
    ]
    ifPrintTemp(1, "Transposing nested sublists of coefficients")
    c_trans = [map(list, zip(*l)) for l in c_lists]
    ifPrintTemp(1, "Flattening nested sublists of coefficients")
    c_flats = [(coeff for il in mat for coeff in il) for mat in c_trans]
    ifPrintTemp(1, "Truncating flattened sublists")
    cutoff = 2 * qNorm(N) * EulerPhi(2,N) / (q^2-1)
    c_cuts = [list(l)[:cutoff] for l in c_flats]
    # c_cuts = [list(l) for l in c_flats]
    ifPrintTemp(1, "Creating final matrix of coefficients")
    mat = matrix(c_cuts)
    r, c = mat.nrows(), mat.ncols()
    ifPrintTemp(1, f"Matrix has dimensions {r}×{c}")

    ifPrintTemp(1, "Calculating full rank of weight 2 D-modular forms")
    W2rank = qNorm(N) * EulerPhi(2,N) / (q^2-1) + EulerPhi(2,N) / (q-1)
    ifPrintTemp(1, f"Full rank of weight 2 D-modular forms is {W2rank}\n")

    ifPrintTemp(1, "Calculating rank of the matrix of coefficients")
    rank = mat.rank()
    ifPrintTemp(1, f"Rank of matrix is {rank}\n")

    n_nonzero = sum(map(lambda row: sum(map(lambda entry: entry != 0, row)), mat))

    end = time.time()


    return {
        "q": q,
        "N": N,
        "N, factored": N.factor(),
        "number of cusps": len(biCusps(N)),
        "finalMatrix": mat,
        "number of nonzero entries": n_nonzero,
        "fraction of nonzero entries": n_nonzero / (mat.nrows()*mat.ncols()),
        "rank": rank,
        "full rank of weight 2 D-modular forms": W2rank
        "time taken": str(datetime.timedelta(seconds=end-start))
    }

SyntaxError: invalid syntax (<ipython-input-81-46db81fdf137>, line 63)

# Results

## $q = 2$, $\deg N = 2$

In [79]:
stats(T^2)

Full rank of weight 2 D-modular forms is 28
Rank of matrix is 27


{'q': 2,
 'N': T^2,
 'N, factored': T^2,
 'number of cusps': 12,
 'finalMatrix': 120 x 36 dense matrix over Function field in e defined by e^2 + T*e + T,
 'number of nonzero entries': 732,
 'fraction of nonzero entries': 0.16944444444444445,
 'rank': 27,
 'full rank of weight 2 D-modular forms': 28}

In [80]:
stats(T^2+T)

Full rank of weight 2 D-modular forms is 21
Rank of matrix is 15


{'q': 2,
 'N': T^2 + T,
 'N, factored': T * (T + 1),
 'number of cusps': 9,
 'finalMatrix': 120 x 27 dense matrix over Function field in e defined by e + 1,
 'number of nonzero entries': 549,
 'fraction of nonzero entries': 0.16944444444444445,
 'rank': 15,
 'full rank of weight 2 D-modular forms': 21}

## $q = 2$, $\deg N = 3$

In [75]:
stats(T^3)

Full rank of weight 2 D-modular forms is 176
Rank of matrix is 176


{'q': 2,
 'N': T^3,
 'N, factored': T^3,
 'number of cusps': 48,
 'finalMatrix': 2016 x 288 dense matrix over Function field in e defined by e^4 + (T^2 + T)*e^2 + T^2*e + T,
 'number of nonzero entries': 91728,
 'fraction of nonzero entries': 0.1579861111111111,
 'rank': 176,
 'full rank of weight 2 D-modular forms': 176}

In [76]:
stats(T^3+1)

Full rank of weight 2 D-modular forms is 165
Rank of matrix is 165


{'q': 2,
 'N': T^3 + 1,
 'N, factored': (T + 1) * (T^2 + T + 1),
 'number of cusps': 45,
 'finalMatrix': 2016 x 270 dense matrix over Function field in e defined by e^3 + (T + 1)*e^2 + T*e + 1,
 'number of nonzero entries': 85995,
 'fraction of nonzero entries': 0.1579861111111111,
 'rank': 165,
 'full rank of weight 2 D-modular forms': 165}

In [77]:
stats(T^3+T^2)

Full rank of weight 2 D-modular forms is 132
Rank of matrix is 132


{'q': 2,
 'N': T^3 + T^2,
 'N, factored': (T + 1) * T^2,
 'number of cusps': 36,
 'finalMatrix': 2016 x 216 dense matrix over Function field in e defined by e^2 + T*e + 1,
 'number of nonzero entries': 68508,
 'fraction of nonzero entries': 0.15732473544973544,
 'rank': 132,
 'full rank of weight 2 D-modular forms': 132}

In [78]:
stats(T^3+T^2+1)

Full rank of weight 2 D-modular forms is 231
Rank of matrix is 231


{'q': 2,
 'N': T^3 + T^2 + 1,
 'N, factored': T^3 + T^2 + 1,
 'number of cusps': 63,
 'finalMatrix': 2016 x 378 dense matrix over Function field in e defined by e^7 + (T^4 + T^2 + T + 1)*e^3 + (T^4 + T^3 + T)*e + T^3 + T^2 + 1,
 'number of nonzero entries': 120897,
 'fraction of nonzero entries': 0.15864748677248677,
 'rank': 231,
 'full rank of weight 2 D-modular forms': 231}

## $q = 3$

In [38]:
stats(T^2)

Full rank of weight 2 D-modular forms is 117
Rank of matrix is 116


{'N': T^2,
 'q': 3,
 'N, factored': T^2,
 'number of cusps': 36,
 'finalMatrix': 3240 x 180 dense matrix over Function field in e defined by e^6 + 2*T*e^4 + T^2*e^2 + T,
 'number of nonzero entries': 87660,
 'fraction of nonzero entries': 0.15030864197530863,
 'rank': 116,
 'full rank of weight 2 D-modular forms': 117}

In [39]:
stats(T^2+T)

Full rank of weight 2 D-modular forms is 104
Rank of matrix is 93


{'N': T^2 + T,
 'q': 3,
 'N, factored': T * (T + 1),
 'number of cusps': 32,
 'finalMatrix': 3240 x 160 dense matrix over Function field in e defined by e^4 + (T + 2)*e^2 + 1,
 'number of nonzero entries': 77920,
 'fraction of nonzero entries': 0.15030864197530863,
 'rank': 93,
 'full rank of weight 2 D-modular forms': 104}

In [43]:
stats(T^2+1)

Full rank of weight 2 D-modular forms is 130
Rank of matrix is 130


{'q': 3,
 'N': T^2 + 1,
 'N, factored': T^2 + 1,
 'number of cusps': 40,
 'finalMatrix': 3240 x 200 dense matrix over Function field in e defined by e^8 + (T^3 + T)*e^2 + T^2 + 1,
 'number of nonzero entries': 97400,
 'fraction of nonzero entries': 0.15030864197530863,
 'rank': 130,
 'full rank of weight 2 D-modular forms': 130}

# Rough Work

In [87]:
m = matrix(QQ,3,3,range(9))
m.kernel()

Vector space of degree 3 and dimension 1 over Rational Field
Basis matrix:
[ 1 -2  1]

In [88]:
m.kernel().basis_matrix()

[ 1 -2  1]

In [89]:
m.kernel().basis_matrix().pivots()

(0,)